In [1]:
TANGO_HOST="192.168.128.177:10000"
visibility_ip = "10.50.1.31"
dish_ids = ["SKA001"]

In [2]:
from PyTango import DeviceProxy, AttributeProxy
import os
import json
from time import sleep
from datetime import datetime

os.environ["TANGO_HOST"] = TANGO_HOST

csp_controller ="mid-csp/control/0"
csp_server = "mid-csp/subarray/01"
csp_controller_dp = DeviceProxy(csp_controller)
csp_dp = DeviceProxy(csp_server)

print(csp_controller_dp.get_command_list())
sleep(5)
csp_controller_dp.adminMode = 0
sleep(3)

with open(f'../../data/mid_telescope/cbf/sys_params/load_dish_config.json', 'r') as json_data:
    d = json.load(json_data)
    csp_controller_dp.LoadDishCfg(json.dumps(d))
    
print(csp_controller_dp.sourceDishVccConfig)
csp_controller_dp.cbfSimulationMode = 0
csp_controller_dp.commandTimeout = 100
csp_controller_dp.On([])


## WAIT 100 Seconds

['AbortCommands', 'CheckLongRunningCommandStatus', 'DebugDevice', 'ExecutePendingOperations', 'GetVersionInfo', 'Init', 'IsCapabilityAchievable', 'LoadDishCfg', 'Off', 'On', 'Reset', 'Standby', 'State', 'Status']



[array([2], dtype=int32), ['1713398512.9165525_138523281600716_On']]

In [3]:
with open(f'../../data/mid_telescope/csp/assign_resources.json', 'r') as json_data:
    assign_resources_csp = json.load(json_data)
    assign_resources_csp["dish"]["receptor_ids"] = dish_ids
    print(assign_resources_csp)
    csp_dp.AssignResources(json.dumps(assign_resources_csp))

sleep(3)

# print(csp_dp.obsState)

with open(f'../../data/mid_telescope/tmc/configure_scan.json', 'r') as json_data:
    d = json.load(json_data)
    csp_configure_scan = d["csp"]
    csp_configure_scan["cbf"]["fsp"][0]["output_host"]=[[]]
    csp_configure_scan["cbf"]["fsp"][0]["output_host"][0]=[0, visibility_ip]
    csp_configure_scan["cbf"]["fsp"][0]["output_port"]=[[0, 21000, 1]]
    print(csp_configure_scan)
    csp_dp.configure(json.dumps(csp_configure_scan))

sleep(5)
print(csp_dp.obsState)

{'interface': 'https://schema.skao.int/ska-csp-assignresources/2.2', 'subarray_id': 1, 'dish': {'receptor_ids': ['SKA001']}}
{'interface': 'https://schema.skao.int/ska-csp-configure/2.3', 'subarray': {'subarray_name': '4 receptors'}, 'common': {'config_id': '4 receptor, band 1, 1 FSP, no options', 'frequency_band': '1', 'subarray_id': 1}, 'cbf': {'fsp': [{'fsp_id': 1, 'function_mode': 'CORR', 'frequency_slice_id': 1, 'zoom_factor': 1, 'zoom_window_tuning': 450000, 'integration_factor': 10, 'channel_offset': 14880, 'output_link_map': [[0, 1]], 'output_host': [[0, '10.50.1.31']], 'output_port': [[0, 21000, 1]]}]}}
obsState.FAULT


In [ ]:
print(csp_dp.obsState)

In [ ]:
now = datetime.now()

with open(f'../../data/mid_telescope/tmc/delay_model.json', 'r') as json_data:
    delay_model = json.load(json_data)

delay_model["epoch"] = float(int(now.timestamp()))

print(delay_model)

tm_server = "ska_mid/tm_leaf_node/csp_subarray_01"
tm_dp = DeviceProxy(tm_server)
tm_dp.delayModel = json.dumps(delay_model)

sleep(5)

with open(f'../../data/mid_telescope/tmc/scan.json', 'r') as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    csp_dp.scan(json.dumps(csp_scan))

sleep(2)
print(csp_dp.obsState)

In [ ]:
csp_dp.EndScan()

In [ ]:
csp_dp.GoToIdle()
sleep(2)
csp_dp.ReleaseAllResources()